# Rpy2 Test
Try this package out, it seems to be the most popular. Instead of calling entire R scripts from the python code this provides a pythonic way to access R packages which will make development easier

In [1]:
%reset -f

In [2]:
import rpy2
import rpy2.robjects as robjects
import rpy2.robjects.packages as rpackages
from rpy2.robjects.vectors import StrVector

from rpy2.robjects import r, pandas2ri
pandas2ri.activate()

Note that in python script we will want to use a path based on file code was run from using something more like 
`dir_path = os.path.dirname(os.path.realpath(__file__))`

In [3]:
import os
cwd = os.getcwd()
data_dir = cwd.rsplit('/', 1)[0] +'/data'

In [5]:

data = robjects.r(f'read.csv("{data_dir}/HVS_data.csv", header=TRUE)')
data.head()

,recid,boro,uf1_1,uf1_3,uf1_4,uf1_5,uf1_6,uf1_7,uf1_8,uf1_9,...,fw74,fw75,fw76,fw77,fw78,fw79,fw80,il30per,il50per,il80per
1,1,1,9,9,9,1,9,9,9,9,...,37635401,33263895,10348898,11446240,32161257,9745805,13739972,21800,36250,58000
2,1,1,9,9,9,1,9,9,9,9,...,8270628,23614216,42797335,19594333,8525261,36330659,28940264,27200,45300,72500
3,1,1,9,9,9,1,9,9,9,9,...,8270628,23614216,42797335,19594333,8525261,36330659,28940264,24500,40800,65250
4,1,1,9,9,9,1,9,9,9,9,...,35934838,10027288,46175949,57167298,43401081,10664017,66729558,21800,36250,58000
5,1,1,9,9,9,1,9,9,9,9,...,50290497,10630692,32310018,70148673,42706530,10137630,64932695,21800,36250,58000


In [6]:
type(data)

pandas.core.frame.DataFrame

Ok sick could load in pandas from .pkl to see if they match?  Save that for later I think

For now do data cleaning in python instead of R

In [7]:
weights_correction = 10**5
data['fw'] = data['fw']/weights_correction

In [8]:
rep_weights_columns = [f'fw{i}' for i in range(1,81)]
data[rep_weights_columns] = data[rep_weights_columns]/weights_correction

Add occupancy final column

In [9]:
data['occ_final'] = data['recid']==1

Use survey package

In [10]:
survey_package = rpackages.importr('survey')
base = rpackages.importr('base')

In [11]:
survey_package = rpackages.importr('survey')
hu_design = survey_package.svrepdesign(variables=data.iloc[:, :188],
                          repweights=data[rep_weights_columns],
                          weights=data['fw'], combined_weights = True, type='other',
                         scale=4/80, rscales=1)

In [13]:
occ_subset = base.subset(hu_design, data['occ_final']==1)
survey_total = survey_package.svytotal(data['occ_final'], design=occ_subset)


In [14]:
survey_total

array([3109954.62348])

In [17]:
data[['occ_final']]

,occ_final
1,True
2,True
3,True
4,True
5,True
...,...
13262,True
13263,True
13264,True
13265,True


Ok correct total but no standard error. Close but not there yet. What happens when we group by sub borough area?

In [ ]:
by_sub_borough_area = survey_package.svyby(formula=data['occ_final'], by=data[['boro','cd']], 
                                           design=hu_design, FUN=survey_package.svytotal)

Ok same measurements of total housing units as R but different standard errors. 

In [23]:
import numpy as np
np.isclose(by_sub_borough_area['V1'].sum(), survey_total[0])

True

Try and get SE by grouping by a column with single value. Jank but helpful to double check

In [25]:
data['a'] = 'a'

In [26]:
survey_total_with_se = survey_package.svyby(formula=data['occ_final'], by=data['a'], 
                                           design=hu_design, FUN=survey_package.svytotal)
survey_total_with_se

,by,V1,se
a,a,3.109955e+06,8611.514099


Matches page 47 of [official guide to calculating variances](https://www2.census.gov/programs-surveys/nychvs/technical-documentation/variances/2017-NYCHVS-Guide-to-Estimating-Variances.pdf), looks good